# Session-based Recs with Transformers4Rec: RNN - Gated Recurrent Networks

Followed a step by step tutorial:
https://nvidia-merlin.github.io/Transformers4Rec/main/examples/tutorial/index.html

## Imports

In [1]:
import os
import glob
import pandas as pd
import numpy as np

from transformers4rec import tf as tr
import tensorflow as tf
from transformers4rec.tf.ranking_metric import NDCGAt, RecallAt

## Instantiates Schema object from schema file

In [2]:
# define the input file path
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", '../data/')
# define the output file path
OUTPUT_DIR = os.environ.get("OUTPUT_DIR", "../data/sessions_by_day")
# features chosen to train on
chosen_features = ['product_id-list_seq']
from merlin_standard_lib import Schema
# define schema object to pass it to the TabularSeqeunceFeatures class
SCHEMA_PATH = os.path.join(INPUT_DATA_DIR, 'schema.pb')
schema = Schema().from_proto_text(SCHEMA_PATH)
schema = schema.select_by_name(chosen_features)

## Define Input Block

use MLM as the training method

In [3]:
# Input
sequence_length, d_model = 20, 192
# Define input module to process tabular input-features and to prepare masked inputs
inputs = tr.TabularSequenceFeatures.from_schema(
    schema,
    max_sequence_length = sequence_length,
    d_output = d_model,
    masking = 'mlm'
)

2021-12-20 17:25:41.156357: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Build Transformer Block

In [4]:
# define XLNetConfig class and set default parameters for HF XLNet config
transformer_config = tr.XLNetConfig.build(
    d_model = d_model, n_head=4, n_layer=2, total_seq_length=sequence_length
)

# define the model block including: inputs, masking, projection and transformer block.

body = tr.SequentialBlock(
    [inputs,
    tr.MLPBlock([192]),
    tr.TransformerBlock(transformer_config, masking=inputs.masking)]
)

# define the head for to the next item prediction task

head = tr.Head(
    body,
    tr.NextItemPredictionTask(
        weight_tying=True, 
        # hf_format=True, 
        metrics=[NDCGAt(top_ks=[10, 20], labels_onehot=True),RecallAt(top_ks=[10, 20], labels_onehot=True)],
        # loss = tf.keras.losses.CategoricalCrossentropy)
))
# head = tr.Head(
#     body,
#     tr.NextItemPredictionTask(
#         weight_tying=True, 
#         # hf_format=True, 
#         metrics=[NDCGAt_temp(top_ks=[10, 20], labels_onehot=False)]
#         # loss = tf.keras.losses.CategoricalCrossentropy)
# ))

# get the end-to-end Model class

model = tr.Model(head)

## Build Datasets

In [5]:
def pad(
    df: pd.DataFrame
):
    df_padded = pd.DataFrame()
    for column in df.columns:
        df_column = df[column].to_list()
        df_column_padded = tf.keras.preprocessing.sequence.pad_sequences(df_column,maxlen=20,padding='pre',truncating='pre')
        df_column_padded = df_column_padded.tolist()
        df_padded[column] = df_column_padded
    return df_padded

In [6]:
def iterate_over_df(
    ### iterator function as input for the tensorflow generator `from_generator` function
    df: pd.DataFrame
):  
    df['empty_list'] = [[] for _ in range(len(df))]
    def caller():
        for _,j in df.iterrows():
            yield(j['product_id-list_seq']),j['empty_list']
    return caller

In [7]:
def ds_from_df(
    ### generate tensorflow object from dataframe
    df: pd.DataFrame
):
    output_shape_x = (
        tf.TensorShape([None,])
    )
    df = tf.data.Dataset.from_generator(
        iterate_over_df(df),
        output_types=((tf.int32),tf.int32),
        output_shapes = (output_shape_x, tf.TensorShape([None,]))
    )
    return df

In [8]:
# def pad_dataset(
#         ## pad dataset so all session sequence data have length 20
#         df,
#         batch_size: int,
# ):
#         df = df.shuffle(5)
#         df = df.padded_batch(batch_size, padded_shapes = (([20,]),[0,]), padding_values = ((0),0),drop_remainder=True)
#         df = df.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

#         return df

In [21]:
def data_to_dict(
    ### create a dictionary tensor dataframe as input into the model
    df_list: list,
    df_len: int,
    chosen_features: list
):
    df_list = (tf.stack([df_list[i][0] for i in range(df_len)]),tf.stack([df_list[i][-1] for i in range(df_len)]))
    dict_features = {}
    dict_targets = {}
    if len(chosen_features) == 1:
        dict_features[chosen_features[0]] = df_list[0]
        dict_targets['target'] = df_list[-1]
    else:
        for i in range(len(df_list[0]),-1):
            dict_features[chosen_features[i]] = df_list[0][i]
    return (dict_features,df_list[-1])

In [28]:
def get_dataset(
    ### function to call all other functions necessary to build the dataset
    ### to input into the model
    df,
    batch_size,
    df_len
):
    df = pad(df)
    df = ds_from_df(df)
    # df = pad_dataset(df,df_len)
    df = data_to_dict(list(df),df_len,chosen_features)
    ds = tf.data.Dataset.from_tensor_slices(df).batch(batch_size)
    steps = int(np.floor(df_len/batch_size))

    # return ds, steps
    return ds, steps

## Daily Fine-tuning: Training over a time window


### Train the model

In [29]:
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.python.keras.engine import data_adapter

def make_print_data_and_train_step(keras_model):
    original_train_step = keras_model.train_step

    def print_data_and_train_step(original_data):
        data = data_adapter.expand_1d(original_data)
        x, y_true, w = data_adapter.unpack_x_y_sample_weight(data)
        y_pred = keras_model(x, training=True)

        # this is pretty much like on_train_batch_begin
        # K.print_tensor(w, "Sample weight (w) =")
        K.print_tensor(x, "Batch input (x) =")
        tf.print(y_true, "Batch output (y_true) =")
        K.print_tensor(y_pred, "Prediction (y_pred) =")

        result = original_train_step(original_data)

        # add anything here for on_train_batch_end-like behavior

        return result

    return print_data_and_train_step

In [30]:
model.train_step = make_print_data_and_train_step(model)

In [31]:
model.compile(optimizer='adam',run_eagerly=True)

In [32]:
train_batch_size = 256
eval_batch_size = 32

In [33]:
%%time
# window
start_time_window_index = 1
final_time_window_index = 3
# Iterating over days of one week
for time_index in range(start_time_window_index, final_time_window_index):
    # Set data 
    time_index_train = time_index
    time_index_eval = time_index + 1
    train_paths = os.path.join(OUTPUT_DIR, f"{time_index_train}/train.parquet")
    eval_paths = os.path.join(OUTPUT_DIR, f"{time_index_eval}/valid.parquet")

    # Load data
    train_df = pd.read_parquet(train_paths)
    train_df = train_df[['product_id-list_seq']]
    eval_df = pd.read_parquet(eval_paths)
    eval_df = eval_df[['product_id-list_seq']]

    # find length of dataframes for argument into `get_dataset`
    train_len = len(train_df)
    eval_len = len(eval_df)

    # get datasets
    train_dataset, train_steps = get_dataset(train_df, train_batch_size,train_len)
    eval_dataset, eval_steps = get_dataset(eval_df, eval_batch_size,eval_len)
    
    # Train on day related to time_index 
    print('*'*20)
    print("Launch training for day %s are:" %time_index)
    print('*'*20 + '\n')
    losses = model.fit(train_dataset, epochs=5)
    model.reset_metrics()
    # Evaluate on the following day
    eval_metrics = model.evaluate(eval_dataset, return_dict=True)
    print('*'*20)
    print("Eval results for day %s are:\t" %time_index_eval)
    print('\n' + '*'*20 + '\n')
    for key in sorted(eval_metrics.keys()):
        print(" %s = %s" % (key, str(eval_metrics[key])))

********************
Launch training for day 1 are:
********************

Epoch 1/5


Projecting inputs of NextItemPredictionTask to'64' As weight tying requires the input dimension '192' to be equal to the item-id embedding dimension '64'


Batch input (x) = {'product_id-list_seq': [[0 0 0 ... 6233 6048 6350]
 [0 0 0 ... 0 9741 9519]
 [0 0 0 ... 29351 14652 14441]
 ...
 [0 0 0 ... 105848 95265 18305]
 [0 0 0 ... 976 785 786]
 [0 0 0 ... 1054 1061 1054]]}
[
 ...] Batch output (y_true) =
Prediction (y_pred) = [[-13.4737015 -14.1514454 -11.4903259 ... -13.180891 -11.1352777 -12.9545994]
 [-12.4430065 -13.0458078 -11.1109505 ... -12.222084 -11.1887035 -11.9529114]
 [-12.1449461 -11.7348089 -10.7625647 ... -12.7315903 -11.7500668 -13.2225733]
 ...
 [-12.5837574 -12.4349346 -11.2143173 ... -12.5857983 -12.0769863 -13.1843815]
 [-11.2788887 -12.6781788 -11.4846125 ... -12.5241127 -11.5999088 -12.0866871]
 [-12.0219593 -11.0468311 -11.8455086 ... -12.6622982 -11.193778 -11.563055]]
Batch input (x) = {'product_id-list_seq': [[0 0 0 ... 6233 6048 6350]
 [0 0 0 ... 0 9741 9519]
 [0 0 0 ... 29351 14652 14441]
 ...
 [0 0 0 ... 105848 95265 18305]
 [0 0 0 ... 976 785 786]
 [0 0 0 ... 1054 1061 1054]]}
[
 ...] Batch output (y_true) =
Pr

KeyboardInterrupt: 